In [44]:
%load_ext autoreload
%autoreload 2

import json
from pathlib import Path

from langchain.prompts import ChatPromptTemplate
import pandas as pd

from ypl.backend.llm.chat import get_chat_model, ModelInfo


template = ChatPromptTemplate.from_messages(
    [("human", """You are a model that will give very concise responses.
IMPORTANT: don't add any explanations on the answer;
don't write full sentences, unless the user is very specifically asking you for a long answer;
for answers that are non-factual, make it witty or funny, but still brief;
if the user asks to output markdown or any markup, return the cleaned text only;
do not use newlines;
NEVER prompt for more information, feedback, or responses.
Respond in fewer than 160 characters, in the language of the user's message.

Here are some examples: 

Question: Why is the sky blue? 
Answer: Rayleigh scattering of sunlight by the atmosphere.

Question: what is the meaning of life?
Answer: 42

Question: How many people are there in the US? 
Answer: 333.3 million

Question: Should I buy Elden Ring?
Answer: Only if you enjoy gorgeous landscapes and repeatedly dying in them
      
Respond to this prompt: {prompt}""")]
)


data = json.loads(Path("../qt-gen.json").read_text())
llms = []

for llm in data["eval_llms"]:
    model_info = ModelInfo(**llm["info"])
    llms.append((model_info, template | get_chat_model(model_info)))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import random

df = pd.read_csv("allall.tsv", sep="\t")
df["single_cat"] = df.category.apply(lambda x: random.choice(eval(x)))

In [46]:
cdf = df.groupby("single_cat").count()
df = df[df.single_cat.isin(set(cdf[cdf.prompt > 20].reset_index().single_cat))]

In [61]:
import asyncio
from tqdm import tqdm

response_data = []

for prompt in tqdm(df.groupby("single_cat").head(15).prompt):
    data = dict()
    (m1, llm1), (m2, llm2) = random.sample(llms, 2)

    try:
        data["model1"], data["model2"] = m1.model, m2.model
        data["response1"], data["response2"] = await asyncio.gather(llm1.ainvoke({"prompt": prompt}), llm2.ainvoke({"prompt": prompt}))
        data["prompt"] = prompt
    except Exception as e:
        continue

    response_data.append(data)


  0%|          | 0/555 [00:00<?, ?it/s]

100%|██████████| 555/555 [1:06:34<00:00,  7.20s/it]   


In [62]:
import torch

torch.save(response_data, "response_data.pt")

In [66]:
data = []

for data_ in response_data:
    d = data_.copy()
    d['response1'] = data_["response1"].content
    d['response2'] = data_["response2"].content    
    data.append(d)


In [67]:
pd.DataFrame(data).to_csv("response_data.tsv", index=False, sep="\t")